In [ ]:
import sys 
sys.path.append('..')
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import torch
import numpy as np
from utils import losses
import pytorch_lightning as pl

from architectures import AR, GRU, LSTM, TGCN

## Plotting packages
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "plotly_white"

In [ ]:
## download 

In [ ]:
df_gru = pd.read_csv("../predictions/predictions_GRU_laced-hill-568.csv", parse_dates=['Date'])
df_gru_pnll = pd.read_csv("../predictions/predictions_GRU_wild-sweep-3.csv", parse_dates=['Date'])
#df_lstm = df_lstm.drop(columns='Unnamed: 0')
df_gru = df_gru.rename(columns={'WEBSTER_pred':'GRU_pred'})
df_gru['GRU_PNLL_pred'] = df_gru_pnll['WEBSTER_pred']

In [ ]:
df_tgcn = pd.read_csv("../predictions/predictions_tgcn_sparkling-leaf-4.csv", parse_dates=['Date'], index_col=0)
df_tgcn = df_tgcn.rename(columns={'WEBSTER_pred':'TGCN_pred'})
df_gru['TGCN_pred'] = df_tgcn['TGCN_pred']

In [ ]:
plot_template = dict(
    layout=go.Layout({
        "font_size": 12,
        "xaxis_title_font_size": 14,
        "yaxis_title_font_size": 14})
    )

df_gru.set_index('Date', inplace=True, drop=True)
fig = px.line(df_gru, labels=dict(created_at="Date", value="Sessions"))
fig.update_layout(
    template=plot_template, legend=dict(orientation='h', y=1.06, title_text="")
)

In [ ]:
df_gru

In [ ]:
import matplotlib.pylab as pylab
params = {'legend.fontsize': 'large',
          'figure.figsize': (15, 5),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)

In [ ]:
start_obs = 600
end_obs = 900
fig, ax = plt.subplots(figsize=(18,11))
ax.plot(df_gru['Date'][start_obs:end_obs], df_gru['WEBSTER_true'][start_obs:end_obs], label = 'True observations')
ax.plot(df_gru['Date'][start_obs:end_obs], df_gru['GRU_pred'][start_obs:end_obs], label = 'Censor-aware GRU_pred')
ax.plot(df_gru['Date'][start_obs:end_obs], df_gru['GRU_PNLL_pred'][start_obs+24:end_obs+24], label = 'Classic GRU_pred')
#ax.plot(df_gru.index[start_obs:end_obs], df_gru['TGCN_pred'][start_obs:end_obs], label = 'TGCN_pred')
ax.plot(df_gru['Date'][start_obs:end_obs], np.ones(len(df_gru.index[start_obs:end_obs]))*2, label = "Threshold", color = 'gray', alpha = 0.7)
ax.legend()
ax.set_xlabel('Date')
ax.set_ylabel('Charging events')
plt.plot()